In [3]:
import open3d as o3d
import os

input_path = 'D:\WorkData\share\ScanNet\scene_raw/'
output_path = 'D:/WorkData/NRSC/output/scene_raw/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

filenamelist = os.listdir(input_path)
filenamelist = [file for file in filenamelist if file.endswith('.ply')]

filelist = []
for file in filenamelist:
    filelist.append(os.path.join(input_path, file))

### 使用PCA预估

In [4]:
# 输出pcd文件为ply,包含点云坐标和法向量
def write_xyznxnynz(pcd,filename):
    xyz = pcd.points
    nxnynz = pcd.normals
    # 输出ply格式文件
    with open(filename, 'w') as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex %d\n" % len(xyz))
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property float nx\n")
        f.write("property float ny\n")
        f.write("property float nz\n")
        f.write("end_header\n")
        for i in range(len(xyz)):
            f.write("%f %f %f %f %f %f\n" % (xyz[i][0], xyz[i][1], xyz[i][2], nxnynz[i][0], nxnynz[i][1], nxnynz[i][2]))
    print('Done:', filename)
    

pcaoutpath = output_path+'\\'+'pca'
hoppeoutpath = output_path+'\\'+'hoppe'
orioutpath = output_path+'\\'+'ori'
if not os.path.exists(pcaoutpath):
    os.makedirs(pcaoutpath)
if not os.path.exists(hoppeoutpath):
    os.makedirs(hoppeoutpath)

for i in range(len(filelist)):
    print(i, filelist[i])
    pcd = o3d.io.read_point_cloud(filelist[i])
    write_xyznxnynz(pcd, orioutpath+'\\'+filenamelist[i])
    pcd.normals = o3d.utility.Vector3dVector([])
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=10))
    write_xyznxnynz(pcd, pcaoutpath+'\\'+filenamelist[i])
    pcd.orient_normals_consistent_tangent_plane(10)
    write_xyznxnynz(pcd, hoppeoutpath+'\\'+filenamelist[i])
    print('Done:', i)
print('All Done')
    
    

0 D:\WorkData\share\ScanNet\scene_raw/scene0000_00_vh_clean.ply


FileNotFoundError: [Errno 2] No such file or directory: 'D:/WorkData/NRSC/output/scene_raw/\\ori\\scene0000_00_vh_clean.ply'

### 得到三种不同初值下的结果

In [ ]:
exe_path = "D:/Work/CG/myProject/NRSC_OP/NRSC-OP-EXE/normal_orientation_propagation_for_NRSC.exe"

ori_nrsc_path = output_path + '/ori_nrsc/'
if not os.path.exists(ori_nrsc_path):
    os.makedirs(ori_nrsc_path)

pca_nrsc_path = output_path + '/pca_nrsc/'
if not os.path.exists(pca_nrsc_path):
    os.makedirs(pca_nrsc_path)
hoppe_nrsc_path = output_path + '/hoppe_nrsc/'
if not os.path.exists(hoppe_nrsc_path):
    os.makedirs(hoppe_nrsc_path)

# 并行处理
for i in range(len(filelist)):
    ori_path = filelist[i]
    pca_path = pcaoutpath+'\\'+filenamelist[i]
    hoppe_path = hoppeoutpath+'\\'+filenamelist[i]
    
    cmd = exe_path+' --in '+pca_path+' --out '+ pca_nrsc_path + filenamelist[i]
    print(cmd)
    os.system(cmd)
    
    cmd = exe_path+' --in '+hoppe_path+' --out '+ hoppe_nrsc_path + filenamelist[i]
    print(cmd)
    os.system(cmd)
    
    cmd = exe_path+' --in '+ori_path+' --out '+ ori_nrsc_path + filenamelist[i]
    print(cmd)
    os.system(cmd)
    print('Done:', i)

In [ ]:
import numpy as np
def cal_loss(pcd_gt, pcd):
    normals_gt = np.asarray(pcd_gt.normals)
    normals = np.asarray(pcd.normals)
    loss = np.sum(np.abs(normals_gt - normals))
    return loss

for filename in filelist:
    pcd_gt = o3d.io.read_point_cloud(input_path+'\\'+filename)
    pcd_pca = o3d.io.read_point_cloud(pcaoutpath+'\\'+filename)
    pcd_hoppe = o3d.io.read_point_cloud(hoppeoutpath+'\\'+filename)
    pcd_ori_nrsc = o3d.io.read_point_cloud(ori_nrsc_path+filename)
    pcd_pca_nrsc = o3d.io.read_point_cloud(pca_nrsc_path+filename)
    pcd_hoppe_nrsc = o3d.io.read_point_cloud(hoppe_nrsc_path+filename)
    
    loss_pca = cal_loss(pcd_gt, pcd_pca)
    loss_hoppe = cal_loss(pcd_gt, pcd_hoppe)
    loss_ori_nrsc = cal_loss(pcd_gt, pcd_ori_nrsc)
    loss_pca_nrsc = cal_loss(pcd_gt, pcd_pca_nrsc)
    loss_hoppe_nrsc = cal_loss(pcd_gt, pcd_hoppe_nrsc)
    
    print(filename + ' loss_pca:' + str(loss_pca) + ' loss_hoppe:' + str(loss_hoppe) + ' loss_ori_nrsc:' + str(loss_ori_nrsc) + ' loss_pca_nrsc:' + str(loss_pca_nrsc) + ' loss_hoppe_nrsc:' + str(loss_hoppe_nrsc))
    
print('All Done')
    
